In [18]:
import json
import pandas as pd
import numpy as np
import os

In [87]:
dataFile='./data/Aaron697_Dickens475_8c95253e-8ee8-9ae8-6d40-021d702dc78e.json'
with open(dataFile,'rb') as file:
    data=file.read()
    data=json.loads(data)

In [88]:
norm_df=pd.json_normalize(data['entry'][0]['resource'])

In [89]:
norm_df

,resourceType,id,extension,identifier,name,telecom,gender,birthDate,deceasedDateTime,address,multipleBirthBoolean,communication,meta.profile,text.status,text.div,maritalStatus.coding,maritalStatus.text
0,Patient,8c95253e-8ee8-9ae8-6d40-021d702dc78e,[{'url': 'http://hl7.org/fhir/us/core/Structur...,[{'system': 'https://github.com/synthetichealt...,"[{'use': 'official', 'family': 'Dickens475', '...","[{'system': 'phone', 'value': '555-152-6034', ...",male,1944-08-28,1998-08-15T13:05:53+01:00,[{'extension': [{'url': 'http://hl7.org/fhir/S...,False,[{'language': {'coding': [{'system': 'urn:ietf...,[http://hl7.org/fhir/us/core/StructureDefiniti...,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",[{'system': 'http://terminology.hl7.org/CodeSy...,M


In [132]:

# norm_df_next=pd.json_normalize(norm_df['extension'][0])

def normalizeListValues(inpVal:list, header:str, attribute:list=[], value:list=[]):
    try:
        print(header)
        # print('inp-',inpVal)
        normDF=pd.json_normalize(inpVal)
        # print(normDF)
        if len(normDF.columns.values)==0:
            # print(col)
            for i in range(len(inpVal)):
                attribute.append(header+'.'+str(i))
                value.append(inpVal[i])
        # print(normDF)
    except Exception as e:
        print('Exception:',e)
        print(inpVal)
        return None,None
    for idx in normDF.index.values:
        for col in normDF.columns.values:
            # print(type(normDF[col][idx]))
            if not isinstance(normDF[col][idx],list) and not pd.isna(normDF[col][idx]):
                attribute.append(header+'.'+str(idx)+'.'+col)
                value.append(normDF[col][idx])
            elif isinstance(normDF[col][idx],list):
                # print(normDF[col][idx])
                attribute, value=normalizeListValues(inpVal=normDF[col][idx], header=header+'.'+str(idx)+'.'+col, attribute=attribute, value=value)
            else:
                print('else')
                print(normDF[col][idx])
    # print(header)
    # print(attribute)
    return attribute, value



In [134]:
attributeData=[]
valueData=[]
resourceIdData=[]
fullUrlData=[]
requestData=[]
for entry in data['entry']:
    df=pd.json_normalize(entry['resource'])
    # print(df.columns.values)
    # resourceId=df['id']
    attribute=[]
    value=[]
    for col in df.columns.values:
        if isinstance(df[col][0],list):
            # print('list-',col)
            attribute, value=normalizeListValues(inpVal=df[col][0], header=col, attribute=attribute, value=value)
        else:
            # print(col)
            attribute.append(col)
            value.append(df[col][0])
    attributeData.extend(attribute)
    valueData.extend(value)
    resourceIdData.extend(list(np.repeat(df['id'][0],len(attribute))))
    fullUrlData.extend(list(np.repeat(entry['fullUrl'],len(attribute))))
    requestData.extend(list(np.repeat(entry['request'],len(attribute))))
    break



extension
extension.0.extension
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
extension.1.extension
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
else
nan
identifier
else
nan
else
nan
identifier.1.type.coding
identifier.2.type.coding
identifier.3.type.coding
identifier.4.type.coding
name
name.0.given
name.0.prefix
telecom
address
address.0.extension
address.0.extension.0.extension
address.0.line
communication
communication.0.language.coding
meta.profile
maritalStatus.coding


In [135]:
outputDF=pd.DataFrame({'RESOURCEID':resourceIdData,'FULLURL':fullUrlData,'REQUEST':requestData,'ATTRIBUTE':attributeData,'VALUE':valueData})

In [136]:
outputDF.to_csv('output_1file.csv',index=False)

In [19]:
sourceFolder='./data/'
sourceFiles=os.listdir(sourceFolder)

In [23]:
sourceFiles=[sourceFolder+file for file in sourceFiles]

In [28]:
dfLen=[]
valTypes=[]
for file in sourceFiles:
    with open(file,'rb') as f:
        data=json.load(f)
    for i in data['entry']:
        norm_df=pd.json_normalize(i['resource'])
        dfLen.append(len(norm_df))
        for col in norm_df.columns.values:
            valTypes.append(type(norm_df[col][0]))